In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
from datetime import datetime as dt, timedelta
from pandas_profiling import ProfileReport
plt.figure(num=None, figsize=(10, 8), dpi=80, facecolor='w', edgecolor='k')
import seaborn as sns; sns.set()  # for plot styling
from sklearn.pipeline import Pipeline
import plotly.graph_objects as go
import datetime
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
display(HTML("<style>.container { width:100% !important; }</style>"))
from plotly.subplots import make_subplots
import plotly.express as px
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error,r2_score
import math
from sklearn.model_selection import train_test_split
import xlsxwriter
from numpy import asarray
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from matplotlib import pyplot
from openpyxl import load_workbook
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestClassifier
from explainerdashboard import ClassifierExplainer, ExplainerDashboard, RegressionExplainer
import pickle
from sklearn.preprocessing import StandardScaler, RobustScaler
# writer = pd.ExcelWriter('output.xlsx', engine = 'xlsxwriter')
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import gspread_dataframe as gd
SCOPE = ['https://www.googleapis.com/auth/drive']
CREDENTIALS = ServiceAccountCredentials.from_json_keyfile_name('../expanded-bonbon-239019-ed390949c71b.json', SCOPE)

In [2]:
def save_model(model, file):
    filename = file + '.sav'
    pickle.dump(model, open(filename, 'wb'))

In [3]:
def plot_results(y_test, predicted, name):
    
    df = pd.DataFrame({'Actual': y_test, 'Predicted': predicted})
    df.to_excel("output.xlsx", sheet_name=name)  
    df1 = df.head(50)
    df1.plot(kind='bar',figsize=(20,12))
    plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
    plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
    return df

In [4]:
def predict(predict_df, scaler, model):
    scaler = pickle.load(open('../models/'+ scaler +'.sav', 'rb'))
    predict_df_scaled = scaler.transform(predict_df)
    predict_df_scaled = pd.DataFrame(data=predict_df_scaled, columns=[predict_df.columns])
    # predict_df_scaled
    xgb = pickle.load(open('../models/'+ model +'.sav', 'rb'))
    predicted = xgb.predict(predict_df_scaled)
    print(predicted)

In [18]:
%store -r fresku_new
fresku_df = fresku_new.copy()

# Scaling

In [6]:
# unaza_e_re_df.loc[unaza_e_re_df.propertiesid == 269740]
# unaza_e_re_df.loc[unaza_e_re_df.bathrooms > unaza_e_re_df.rooms]

In [7]:
# unaza_e_re_df = unaza_e_re_df.loc[unaza_e_re_df.gros_area < 240]
# unaza_e_re_df

In [10]:
# unaza_e_re_df.drop('gros_area', 1, inplace=True)

In [19]:
scaler = RobustScaler()

X = fresku_df.drop(['price', 'gros_area','property_status'],1)
y = fresku_df['price']

columns = ['interior_area', 'bathrooms','rooms']
X.set_index('propertiesid', inplace=True)
scaler = scaler.fit(X)

X_scaled = scaler.transform(X)

X_df = pd.DataFrame(X_scaled, columns=columns)

X.reset_index(inplace=True)
properties = X[['propertiesid']]
X_new_df = properties.merge(X_df, left_index=True, right_index=True)
X_new_df.set_index('propertiesid', inplace=True)

In [21]:
save_model(scaler, 'fresku_scaler')

In [22]:
save_model(model, "fresku")

In [50]:
# scaler = RobustScaler()

# X = fresku_df.drop(['price','propertiesid'],1)

# y = fresku_df['price']
# columns = X.columns 

# scaler = scaler.fit(X)
# save_model(scaler, '../models/scaler')
# X = scaler.transform(X)
# X = pd.DataFrame(X, columns=[columns])
# # X_train
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [20]:
from sklearn.ensemble import RandomForestRegressor
from explainerdashboard import ClassifierExplainer, ExplainerDashboard, RegressionExplainer
X_train, X_test, y_train, y_test = train_test_split(X_new_df, y, test_size = 0.20)
model = RandomForestRegressor(n_estimators = 300, max_depth=4)
model.fit(X_train,y_train)
feature_descriptions = {
    "interior_area": "Interior Area of the Apartment",
    "gros_area": "Gross Area of the Apartment",
    "bathrooms": "Number of bathrooms",
    "rooms": "Number of rooms, includes bedrooms and other rooms"}

explainer = RegressionExplainer(model, X_test, y_test, 
       descriptions=feature_descriptions,
       index_name = "propertiesid"# defaults to None
                                )
explainer.dump("randomforest.joblib")
db = ExplainerDashboard(explainer).run()

Changing class type to RandomForestRegressionExplainer...
Generating self.shap_explainer = shap.TreeExplainer(model)
Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating shap values...
Calculating predictions...
Calculating residuals...
Calculating absolute residuals...
Calculating dependencies...
Calculating importances...
Calculating shap interaction values...
Reminder: TreeShap computational complexity is O(TLD^2), where T is the number of trees, L is the maximum number of leaves in any tree and D the maximal depth of any tree. So reducing these will speed up the calculation.
Calculating ShadowDecTree for each individual decision tree...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explaine

 * Running on http://0.0.0.0:8050/ (Press CTRL+C to quit)
192.168.0.117 - - [20/Jul/2021 12:27:48] "GET / HTTP/1.1" 200 -
192.168.0.117 - - [20/Jul/2021 12:27:48] "GET /_dash-dependencies HTTP/1.1" 200 -
192.168.0.117 - - [20/Jul/2021 12:27:48] "GET /_dash-layout HTTP/1.1" 200 -
192.168.0.117 - - [20/Jul/2021 12:27:48] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.117 - - [20/Jul/2021 12:27:48] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.117 - - [20/Jul/2021 12:27:48] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.117 - - [20/Jul/2021 12:27:48] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.117 - - [20/Jul/2021 12:27:48] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.117 - - [20/Jul/2021 12:27:48] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.117 - - [20/Jul/2021 12:27:48] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.117 - - [20/Jul/2021 12:27:48] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.0.117 - - [20/Ju

In [17]:
ExplainerDashboard(explainer).terminate(8050)

Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating dependencies...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Trying to shut down dashboard on port 8050...
Something seems to have failed: HTTPConnectionPool(host='localhost', port=8050): Max retries exceeded with url: /_shutdown_7c7bdc43-1588-405d-9edf-1703f2a642d1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f8a7ba92ee0>: Failed to establish a new connection: [Errno 111] Connection refused'))


# XGBoost

## GridSearch

In [8]:
# xgb1 = XGBRegressor()
# parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
#               'objective':['reg:linear'],
#               'learning_rate': [.03, 0.05, .07], #so called `eta` value
#               'max_depth': [5, 6, 7],
#               'min_child_weight': [4],
#               'silent': [1],
#               'subsample': [0.7],
#               'colsample_bytree': [0.7],
#               'n_estimators': [500]}

# xgb_grid = GridSearchCV(xgb1,
#                         parameters,
#                         cv = 2,
#                         n_jobs = 5,
#                         verbose=True)

# xgb_grid.fit(X_train, y_train)

# print(xgb_grid.best_score_)
# print(xgb_grid.best_params_)

## CrossFold Validation

In [9]:
# model = XGBRegressor(colsample_bytree= 0.7, learning_rate= 0.03, max_depth= 5, min_child_weight= 4, n_estimators= 500, nthread= 4, objective= 'reg:linear', silent= 1, subsample= 0.7)
# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# n_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
# print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [12]:
# model = XGBRegressor(colsample_bytree= 0.7, learning_rate= 0.03, max_depth= 5, min_child_weight= 4, n_estimators= 500, nthread= 4, objective= 'reg:linear', silent= 1, subsample= 0.7)
# model.fit(X_train, y_train)
# save_model(model, "../models/xgbregressor")

In [13]:
# predicted = model.predict(X_test)
# print('R Squared:', metrics.r2_score(y_test, predicted))
# print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predicted))  
# print('Mean Squared Error:', metrics.mean_squared_error(y_test, predicted))  
# print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predicted)))

In [14]:
# predict_df = pd.DataFrame(data=[{'interior_area':70, 'gros_area':78,'bathrooms':1, 'rooms':2}])
# predict(predict_df,'scaler','xgbregressor')

In [ ]:
# plot_results(y_test, predicted,'XGBoost')

# Random Forest Regressor

In [44]:
from sklearn.ensemble import RandomForestRegressor

In [16]:
parameters = {
    'n_estimators': [100, 150, 200, 250, 300],
    'max_depth': [1,2,3,4],
}
regr = RandomForestRegressor(random_state=0)

clf = GridSearchCV(regr, parameters)
clf.fit(X_train, y_train)

print(clf.best_score_)
print(clf.best_params_)

0.6469002570946338
{'max_depth': 4, 'n_estimators': 300}


In [51]:
model_rfr = RandomForestRegressor(n_estimators = 300, max_depth=4)
model_rfr.fit(X_train, y_train)
predicted_rfr = model_rfr.predict(X_test)
# save_model(model_rfr, "../models/rfr")

In [52]:
predicted = model_rfr.predict(X_test)
print('R Squared:', metrics.r2_score(y_test, predicted))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predicted))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predicted))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predicted)))

R Squared: 0.7015314104122812
Mean Absolute Error: 8189.588330722166
Mean Squared Error: 134719830.41873834
Root Mean Squared Error: 11606.887197639957


# ExplainerDashboard RandomForestClassifier

In [45]:
# fresku_df.drop('gros_area', 1, inplace=True)

In [53]:
scaler = RobustScaler()

X = fresku_df.drop('price',1)
y = fresku_df['price']

columns = ['gros_area','interior_area', 'bathrooms','rooms']
X.set_index('propertiesid', inplace=True)
scaler = scaler.fit(X)
# save_model(scaler, '../models/scaler')
X_scaled = scaler.transform(X)

X_df = pd.DataFrame(X_scaled, columns=columns)

X.reset_index(inplace=True)
properties = X[['propertiesid']]
X_new_df = properties.merge(X_df, left_index=True, right_index=True)
X_new_df.set_index('propertiesid', inplace=True)

In [54]:
from sklearn.ensemble import RandomForestClassifier
from explainerdashboard import ClassifierExplainer, ExplainerDashboard, RegressionExplainer
X_train, X_test, y_train, y_test = train_test_split(X_new_df, y, test_size = 0.20)
model = RandomForestRegressor(n_estimators = 300, max_depth=4)
model.fit(X_train,y_train)
feature_descriptions = {
    "interior_area": "Interior Area of the Apartment",
    "gros_area": "Gross Area of the Apartment",
    "bathrooms": "Number of bathrooms",
    "rooms": "Number of rooms, includes bedrooms and other rooms"}

explainer = RegressionExplainer(model, X_test, y_test, 
       descriptions=feature_descriptions,
       index_name = "propertiesid"# defaults to None
                                )
explainer.dump("randomforest_fresku.joblib")
db = ExplainerDashboard(explainer).run()

Changing class type to RandomForestRegressionExplainer...
Generating self.shap_explainer = shap.TreeExplainer(model)
Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating shap values...
Calculating predictions...
Calculating residuals...
Calculating absolute residuals...
Calculating dependencies...
Calculating importances...
Calculating shap interaction values...
Reminder: TreeShap computational complexity is O(TLD^2), where T is the number of trees, L is the maximum number of leaves in any tree and D the maximal depth of any tree. So reducing these will speed up the calculation.
Calculating ShadowDecTree for each individual decision tree...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explaine

 * Running on http://0.0.0.0:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jun/2021 20:49:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2021 20:49:56] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2021 20:49:56] "GET /assets/favicon.ico?m=1623491275.4921412 HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2021 20:49:56] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2021 20:49:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2021 20:49:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2021 20:49:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2021 20:49:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2021 20:49:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2021 20:49:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2021 20:49:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2021 20:49:57] "POST /_dash-up

In [55]:
ExplainerDashboard(explainer).terminate(8050)

Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating dependencies...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Trying to shut down dashboard on port 8050...
Something seems to have failed: HTTPConnectionPool(host='localhost', port=8050): Max retries exceeded with url: /_shutdown_f3469eb4-f33f-4290-bedf-cfcdbb8f919a (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f6ea7170bb0>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [229]:
# predict_df = pd.DataFrame(data=[{'interior_area':70, 'gros_area':78,'bathrooms':1, 'rooms':2}])
# predict(predict_df,'scaler','rfr')

# ExplainerDashboard XGBRegressor

In [28]:

X_train, X_test, y_train, y_test = train_test_split(X_new_df, y, test_size = 0.20)
model = XGBRegressor(colsample_bytree= 0.7, learning_rate= 0.03, max_depth= 5, min_child_weight= 4, n_estimators= 500, nthread= 4, objective= 'reg:linear', silent= 1, subsample= 0.7)
model.fit(X_train,y_train)
feature_descriptions = {
    "interior_area": "Interior Area of the Apartment",
    "gros_area": "Gross Area of the Apartment",
    "bathrooms": "Number of bathrooms",
    "rooms": "Number of rooms, includes bedrooms and other rooms"}

explainer = RegressionExplainer(model, X_test, y_test, 
       descriptions=feature_descriptions,
       index_name = "propertiesid"# defaults to None
                                )
explainer.dump("xgbregressor.joblib")
db = ExplainerDashboard(explainer).run()

[20:09:58] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[20:09:58] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Changing class type to XGBRegressionExplainer...
Generating self.shap_explainer = shap.TreeExplainer(model)
Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating shap values...
Calculating predictions...
Calculating residuals...
Calculating absolute residuals...
Generating xgboost model dump...
Calculating dependencies...
Calculating importances...
Calculating shap interaction

 * Running on http://0.0.0.0:8050/ (Press CTRL+C to quit)


# Compare results